In [2]:
import pandas as pd
from transformers import RobertaTokenizer, TrainingArguments
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from datasets import DatasetDict,load_dataset,Dataset
from transformers import Trainer
from transformers import RobertaForSequenceClassification
import torch
import numpy as np
import pandas as pd
import time
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

#### Reading data

In [ ]:
dataset = load_dataset('pandas', data_files='imdb.pkl', split="train")
dataset.remove_columns_(column_names = ['__index_level_0__'])

#### Preprocessing

In [6]:
dataset = load_dataset('pandas', data_files='imdb.pkl', split="train")
dataset.remove_columns_(column_names = ['__index_level_0__'])
train_testvalid = dataset.train_test_split(test_size=0.1)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
all_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})
tokenized_datasets = all_dataset.map(tokenize_function, batched=True)
tokenized_datasets

Using custom data configuration default-eaf32cec703c69fd
Reusing dataset pandas (/Users/mgaulia/.cache/huggingface/datasets/pandas/default-eaf32cec703c69fd/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade)


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text'],
        num_rows: 2700
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text'],
        num_rows: 150
    })
    valid: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text'],
        num_rows: 150
    })
})

#### Training

In [7]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = 2)

training_args = TrainingArguments(
    output_dir='./results'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    tokenizer=tokenizer
)
trainer.train()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Step,Training Loss


KeyboardInterrupt: 

#### Saving model and calculating metrics

In [ ]:
trainer.save_model("trained_on_setiment_imbd")

In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])
y_true = tokenized_datasets["test"]["label"]
y_pred = np.argmax(predictions.predictions, axis=-1)
confusion_matrix(y_true, y_pred)

In [ ]:
metric = load_metric("accuracy")
metric.compute(predictions=y_pred, references=y_true)